# QBID under TCJA

## Key Quantities for Section 199A

Let:

- $ \text{QBI} $: Qualified Business Income — net income from a U.S. trade or business
- $ W $: W-2 wages paid
- $ P $: UBIA (Unadjusted Basis Immediately after Acquisition) of qualified property
- $\text{WPA} = \max \left( 0.5 \cdot W,\ 0.25 \cdot W + 0.025 \cdot P \right)$
- $ \theta_{\max} $: maximum deduction rate (typically 0.20)
- $ T $: taxable income  
- $ G $: net capital gain  
- $ \text{MTI} = T - G $: Modified Taxable Income  
- $ \tau $: income threshold  
- $ \ell $: phase-in range  
- $ p = \min \left( 1,\ \max \left( 0,\ \frac{T - \tau}{\ell} \right) \right) $: proportion through the phase-in
- $ \text{RP}$: combined REIT and PTP income

## The Sum Described in (1)(A)

The internal limitation described in Section 199A(b)(2)(B)(i)(I) is:

$$
S^{(1)(A)} = \min \left( \theta_{\max} \cdot \text{QBI},\ \text{WPA} \right)
$$


## non-SSTB Preliminary Deduction (Before MTI Cap)

Let $D_{\text{pre}}$ be the preliminary deduction before applying the MTI limitation:

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\[12pt]
\theta_{\max} \cdot \text{QBI} - p \cdot \left( \theta_{\max} \cdot \text{QBI} - S^{(1)(A)} \right), & \text{if } \tau < T < \tau + \ell \\\\[12pt]
S^{(1)(A)}, & \text{if } T \geq \tau + \ell
\end{cases}
$$



## SSTB Preliminary Deduction (Before MTI Cap)

Let $D_{\text{pre}}$ be the preliminary deduction before applying the MTI limitation:

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\[12pt]
(1 - p) S^{(1)(A)}, & \text{if } \tau < T < \tau + \ell \\\\[12pt]
0, & \text{if } T \geq \tau + \ell
\end{cases}
$$


## Final Deduction (After MTI Cap)

The final deduction is the lesser of the preliminary deduction and the MTI cap:

$$
\text{Deduction}^{\text{TCJA}} = \min \left( D_{\text{pre}},\ \theta_{\max} \cdot \text{MTI} \right) + \theta_{\max} \cdot \text{RP}
$$

# QBID under H.R. 1

In addition to the other parameters, Let:

- $r$: ramp-down rate (75¢ of deduction lost per $1 above threshold τ)
- RPB: combined REIT, PTP, and BDC income
  
---

### Non-SSTB Deduction

**Preliminary deduction (before MTI cap):**

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\
\max \left(S^{(1)(A)},\ \theta_{\max} \cdot \text{QBI} - r \cdot (T - \tau) \right), & \text{if } T > \tau
\end{cases}
$$

---

### SSTB Deduction

**Preliminary deduction (before MTI cap):**

$$
D_{\text{pre}} =
\begin{cases}
\theta_{\max} \cdot \text{QBI}, & \text{if } T \leq \tau \\\\
\max \left( 0,\ \theta_{\max} \cdot \text{QBI} - r \cdot (T - \tau) \right), & \text{if } T > \tau
\end{cases}
$$


## The final H.R. 1 Section 199A Deduction

$$
\text{Deduction}^{\text{H.R.1}} = \min \left( D_{\text{pre}},\ \theta_{\max} \cdot \text{MTI} \right) + \theta_{\max} \cdot \text{RPB}
$$

# Code Examples - QBID under TCJA
## Starter code to run qbid_amount.py interactively

In [16]:
import numpy as np
from numpy import where

from policyengine_us_data.datasets.cps.enhanced_cps import EnhancedCPS_2024

from policyengine_core.periods import period as period_  # alias, "period" will be a var
from policyengine_core.populations.population import Population
from policyengine_core.entities import Entity
from policyengine_core.reforms import Reform

from policyengine_us import Microsimulation
from policyengine_us.model_api import add, max_, min_

# Create a simulation as we will harvest its internals
sim = Microsimulation(dataset=EnhancedCPS_2024)
year = 2021
period = period_(year)

# Entity init arguments are: the key, the plural, the label, the doc string
Person = Entity("person", "people", "Person", "A person")
print(Person.key)

# Population: Lowercase p person
person = Population(Person)
print(f"The person population is a population of {person.entity.plural}")
person.simulation = sim  # populations need simulations
person.entity.set_tax_benefit_system(sim.tax_benefit_system)  # and a tax benefit system through their entity

parameters = sim.tax_benefit_system.parameters

/home/baogorek/envs/pe_linux/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


person
The person population is a population of people


**Entity 1 — Non-SSTB under TCJA**

Entity 1 has qualified business income (QBI) of \\$80,000, resulting in a tentative QBI deduction of \\$16,000 (20% of QBI). The taxpayer’s income of \\$400,100 is \\$60,000 above the threshold of \\$340,100. This places them 60% of the way through the \\$100,000 phase-in range.

For non-SSTBs, the deduction is reduced by a proportion $p$ of the difference between the full deduction and the wage and property amount (WPA), which here is \\$4,000.

$$
\text{Reduction} = p \cdot (\theta_{\max} \cdot \text{QBI} - \text{WPA}) = 0.6 \cdot (16,000 - 4,000) = 7,200
$$

The resulting deduction is:

$$
\text{Deduction} = 16,000 - 7,200 = 8,800
$$

If taxable income were raised to \\$460,100, the full \\$100,000 phase-in would apply, and the deduction would bottom out at the floor:

$$
\text{Deduction} = \text{WPA} = 4,000
$$


In [61]:
is_sstb = 0
qbi = 80_000
w2_wages = 8_000
ubia_property = 50_000
taxable_income = 4000_100
threshold = 340_100
phase_in_range = 100_000
reit_ptp_income = 0

In [62]:
p = parameters(period).gov.irs.deductions.qbi

# 1. Core inputs ---------------------------------------------------------
# Already provided: qbi, w2_wages, ubia_property, taxable_income, threshold, etc.

# 2. 20% of QBI ----------------------------------------------------------
qbid_max = p.max.rate * qbi  # 20% of qualified business income
print("qbid_max:", qbid_max)

# 3. Wage / UBIA limitation ---------------------------------------------
wage_limit = p.max.w2_wages.rate * w2_wages  # 50% of W-2 wages
print("wage_limit:", wage_limit)

alt_limit = (
    p.max.w2_wages.alt_rate * w2_wages       # 25% of W-2 wages
    + p.max.business_property.rate * ubia_property  # 2.5% of UBIA
)
print("alt_limit:", alt_limit)

wage_ubia_cap = max_(wage_limit, alt_limit)
print("wage_ubia_cap:", wage_ubia_cap)

# 4. Phase-in percentage (§199A(b)(3)(B)) --------------------------------
over_threshold = max_(0, taxable_income - threshold)
print("over_threshold:", over_threshold)

phase_in_pct = min_(1, over_threshold / phase_in_range)
print("phase_in_pct:", phase_in_pct)

# 5. Applicable percentage for SSTBs ------------------------------------
applicable_pct = where(is_sstb, 1 - phase_in_pct, 1)
print("applicable_pct:", applicable_pct)

adj_qbid_max = qbid_max * applicable_pct
print("adj_qbid_max:", adj_qbid_max)

adj_cap = wage_ubia_cap * applicable_pct
print("adj_cap:", adj_cap)

limited_deduction = min_(adj_qbid_max, adj_cap)
print("limited_deduction:", limited_deduction)

excess = max_(0, adj_qbid_max - adj_cap)
print("excess:", excess)

phased_deduction = max_(0, adj_qbid_max - phase_in_pct * excess)
print("phased_deduction:", phased_deduction)

# 6. Deduction before income cap ----------------------------------------
deduction_pre_cap = where(
    phase_in_pct == 0,                      # Below lower threshold
    adj_qbid_max,
    where(
        is_sstb,                            # SSTB logic
        limited_deduction,                  #  -> only the scaled min()
        where(                              # Non-SSTB logic
            phase_in_pct < 1,
            phased_deduction,               #  -> reduction formula
            limited_deduction               #  -> fully phased-in
        )
    )
)

print("deduction_pre_cap:", deduction_pre_cap)

# 7. REIT / PTP component (always 20%) ----------------------------------
reit_ptp_deduction = p.max.rate * reit_ptp_income
print("reit_ptp_deduction:", reit_ptp_deduction)

total_before_income_cap = deduction_pre_cap + reit_ptp_deduction
print("total_before_income_cap:", total_before_income_cap)

# 8. Overall 20% income cap (§199A(a)(2)) -------------------------------
income_cap = p.max.rate * taxable_income
print("income_cap:", income_cap)

_return = min_(total_before_income_cap, income_cap)
print("Final QBID (_return):", _return)

qbid_max: 16000.0
wage_limit: 4000.0
alt_limit: 3250.0
wage_ubia_cap: 4000.0
over_threshold: 3660000
phase_in_pct: 1.0
applicable_pct: 1.0
adj_qbid_max: 16000.0
adj_cap: 4000.0
limited_deduction: 4000.0
excess: 12000.0
phased_deduction: 4000.0
deduction_pre_cap: 4000.0
reit_ptp_deduction: 0.0
total_before_income_cap: 4000.0
income_cap: 800020.0
Final QBID (_return): 4000.0


## Code Examples - QBID under H.R. 1

**Entity 1 — Non-SSTB under H.R. 1 (OBBBA)**

The initial deduction is again \$16,000. Under OBBBA, this deduction is reduced by 75¢ for every dollar of income above the threshold:

$$
\text{Reduction} = r \cdot (T - \tau) = 0.75 \cdot 60,000 = 45,000
$$

This would fully eliminate the deduction, but OBBBA sets a floor at the wage and property amount:

$$
\text{Deduction} = \max(\text{WPA},\ \theta_{\max} \cdot \text{QBI} - r \cdot (T - \tau)) = \max(4,000,\ -29,000) = 4,000
$$


In [56]:
is_sstb = 0
qbi = 80_000
w2_wages = 8_000
ubia_property = 50_000
taxable_income = 4000_100
threshold = 340_100
phase_in_range = 100_000
phase_in_range = 100_000
reit_ptp_income = 0
bdc_income = 0

In [57]:
p_ref = parameters(period).gov.contrib.reconciliation.qbid
phase_in_rate = p_ref.phase_out_rate  # 75% ramp-down rate

# 1. Core Inputs --------------------------------------------------------
# qbi: Form 8995, line 1
# is_sstb: SSTB check box
# reit_ptp_income: Form 8995, line 6
# bdc_income: new addition under H.R. 1
# taxable_income: Form 1040, line 15 (before QBID)
# filing status from Form 1040

print("phase_in_rate:", phase_in_rate)

# 2. 23% of total QBI ---------------------------------------------------
qbid_max = p.max.rate * qbi
print("qbid_max (23% of QBI):", qbid_max)

# 3. Wage / UBIA limitation (non-SSTBs only) ----------------------------
qbi_non_sstb = (1 - is_sstb) * qbi
w2_wages_non_sstb = (1 - is_sstb) * w2_wages
ubia_property_non_sstb = (1 - is_sstb) * ubia_property

print("qbi_non_sstb:", qbi_non_sstb)
print("w2_wages_non_sstb:", w2_wages_non_sstb)
print("ubia_property_non_sstb:", ubia_property_non_sstb)

wage_limit = p.max.w2_wages.rate * w2_wages_non_sstb  # 50% of wages
print("wage_limit:", wage_limit)

alt_limit = (
    p.max.w2_wages.alt_rate * w2_wages_non_sstb  # 25% of wages
    + p.max.business_property.rate * ubia_property_non_sstb  # 2.5% of UBIA
)
print("alt_limit:", alt_limit)

wage_ubia_cap = max_(wage_limit, alt_limit)
print("wage_ubia_cap:", wage_ubia_cap)

step1_uncapped = p.max.rate * qbi_non_sstb
print("step1_uncapped:", step1_uncapped)

step1_deduction = min_(step1_uncapped, wage_ubia_cap)
print("step1_deduction:", step1_deduction)

# 4. Limitation phase-in amount (75% × excess) --------------------------
excess_income = max_(0, taxable_income - threshold)
print("excess_income:", excess_income)

phase_in_amount = phase_in_rate * excess_income
print("phase_in_amount (reduction):", phase_in_amount)

step2_deduction = max_(0, qbid_max - phase_in_amount)
print("step2_deduction:", step2_deduction)

# 5. QBI component: greater of Step 1 or Step 2 -------------------------
qbi_component = max_(step1_deduction, step2_deduction)
print("qbi_component:", qbi_component)

# 6. REIT, PTP, and BDC income (always 23%) -----------------------------
reit_ptp_bdc_base = reit_ptp_income + where(
    p_ref.use_bdc_income,
    bdc_income,
    0,
)
print("reit_ptp_bdc_base (RPB):", reit_ptp_bdc_base)

reit_ptp_bdc_deduction = p.max.rate * reit_ptp_bdc_base
print("reit_ptp_bdc_deduction:", reit_ptp_bdc_deduction)

total_before_income_cap = qbi_component + reit_ptp_bdc_deduction
print("total_before_income_cap:", total_before_income_cap)

# 7. Overall 23% taxable income ceiling ---------------------------------
income_cap = p.max.rate * taxable_income
print("income_cap (23% of taxable_income):", income_cap)

_return = min_(total_before_income_cap, income_cap)
print("Final QBID (_return):", _return)

phase_in_rate: 0.75
qbid_max (23% of QBI): 16000.0
qbi_non_sstb: 80000
w2_wages_non_sstb: 8000
ubia_property_non_sstb: 50000
wage_limit: 4000.0
alt_limit: 3250.0
wage_ubia_cap: 4000.0
step1_uncapped: 16000.0
step1_deduction: 4000.0
excess_income: 3660000
phase_in_amount (reduction): 2745000.0
step2_deduction: 0.0
qbi_component: 4000.0
reit_ptp_bdc_base (RPB): 0
reit_ptp_bdc_deduction: 0.0
total_before_income_cap: 4000.0
income_cap (23% of taxable_income): 800020.0
Final QBID (_return): 4000.0


**Entity 5 — SSTB under TCJA**

Entity 5 has \\$110,000 in QBI, producing a tentative deduction of \\$22,000. Since this is an SSTB, the deduction phases out with no floor. The taxpayer's income is again 60% into the \\$100,000 phase-in range:

$$
\text{Deduction} = (1 - p) \cdot (\theta_{\max} \cdot \text{QBI}) = (1 - 0.6) \cdot 22,000 = 8,800
$$

If taxable income were \\$460,100, the deduction would be completely phased out:

$$
\text{Deduction} = 0
$$


In [64]:
# Entity 5 under TCJA (SSTB)
is_sstb = 1
qbi = 110_000
w2_wages = 60_000
ubia_property = 0
wpa = 30_000  # override if needed
taxable_income = 400_100
threshold = 340_100
phase_in_range = 100_000


In [65]:
p = parameters(period).gov.irs.deductions.qbi

# 1. Core inputs ---------------------------------------------------------
# Already provided: qbi, w2_wages, ubia_property, taxable_income, threshold, etc.

# 2. 20% of QBI ----------------------------------------------------------
qbid_max = p.max.rate * qbi  # 20% of qualified business income
print("qbid_max:", qbid_max)

# 3. Wage / UBIA limitation ---------------------------------------------
wage_limit = p.max.w2_wages.rate * w2_wages  # 50% of W-2 wages
print("wage_limit:", wage_limit)

alt_limit = (
    p.max.w2_wages.alt_rate * w2_wages       # 25% of W-2 wages
    + p.max.business_property.rate * ubia_property  # 2.5% of UBIA
)
print("alt_limit:", alt_limit)

wage_ubia_cap = max_(wage_limit, alt_limit)
print("wage_ubia_cap:", wage_ubia_cap)

# 4. Phase-in percentage (§199A(b)(3)(B)) --------------------------------
over_threshold = max_(0, taxable_income - threshold)
print("over_threshold:", over_threshold)

phase_in_pct = min_(1, over_threshold / phase_in_range)
print("phase_in_pct:", phase_in_pct)

# 5. Applicable percentage for SSTBs ------------------------------------
applicable_pct = where(is_sstb, 1 - phase_in_pct, 1)
print("applicable_pct:", applicable_pct)

adj_qbid_max = qbid_max * applicable_pct
print("adj_qbid_max:", adj_qbid_max)

adj_cap = wage_ubia_cap * applicable_pct
print("adj_cap:", adj_cap)

limited_deduction = min_(adj_qbid_max, adj_cap)
print("limited_deduction:", limited_deduction)

excess = max_(0, adj_qbid_max - adj_cap)
print("excess:", excess)

phased_deduction = max_(0, adj_qbid_max - phase_in_pct * excess)
print("phased_deduction:", phased_deduction)

# 6. Deduction before income cap ----------------------------------------
deduction_pre_cap = where(
    phase_in_pct == 0,                      # Below lower threshold
    adj_qbid_max,
    where(
        is_sstb,                            # SSTB logic
        limited_deduction,                  #  -> only the scaled min()
        where(                              # Non-SSTB logic
            phase_in_pct < 1,
            phased_deduction,               #  -> reduction formula
            limited_deduction               #  -> fully phased-in
        )
    )
)

print("deduction_pre_cap:", deduction_pre_cap)

# 7. REIT / PTP component (always 20%) ----------------------------------
reit_ptp_deduction = p.max.rate * reit_ptp_income
print("reit_ptp_deduction:", reit_ptp_deduction)

total_before_income_cap = deduction_pre_cap + reit_ptp_deduction
print("total_before_income_cap:", total_before_income_cap)

# 8. Overall 20% income cap (§199A(a)(2)) -------------------------------
income_cap = p.max.rate * taxable_income
print("income_cap:", income_cap)

_return = min_(total_before_income_cap, income_cap)
print("Final QBID (_return):", _return)

qbid_max: 22000.0
wage_limit: 30000.0
alt_limit: 15000.0
wage_ubia_cap: 30000.0
over_threshold: 60000
phase_in_pct: 0.6
applicable_pct: 0.4
adj_qbid_max: 8800.0
adj_cap: 12000.0
limited_deduction: 8800.0
excess: 0.0
phased_deduction: 8800.0
deduction_pre_cap: 8800.0
reit_ptp_deduction: 0.0
total_before_income_cap: 8800.0
income_cap: 80020.0
Final QBID (_return): 8800.0


**Entity 5 — SSTB under H.R. 1 (OBBBA)**

The full deduction of \$22,000 is again reduced by:

$$
\text{Reduction} = r \cdot (T - \tau) = 0.75 \cdot 60,000 = 45,000
$$

No floor applies to SSTBs, so the deduction becomes:

$$
\text{Deduction} = \max(0,\ 22,000 - 45,000) = 0
$$

If the taxpayer’s income were only \$10,000 above the threshold, the deduction would be:

$$
\text{Deduction} = \max(0,\ 22,000 - 7,500) = 14,500
$$


In [59]:
# Entity 5 under TCJA (SSTB)
is_sstb = 1
qbi = 110_000
w2_wages = 60_000
ubia_property = 0
wpa = 30_000  # override if needed
taxable_income = 400_100
# taxable_income = 350_100
threshold = 340_100
phase_in_range = 100_000

In [60]:
p_ref = parameters(period).gov.contrib.reconciliation.qbid
phase_in_rate = p_ref.phase_out_rate  # 75% ramp-down rate

# 1. Core Inputs --------------------------------------------------------
# qbi: Form 8995, line 1
# is_sstb: SSTB check box
# reit_ptp_income: Form 8995, line 6
# bdc_income: new addition under H.R. 1
# taxable_income: Form 1040, line 15 (before QBID)
# filing status from Form 1040

print("phase_in_rate:", phase_in_rate)

# 2. 23% of total QBI ---------------------------------------------------
qbid_max = p.max.rate * qbi
print("qbid_max (23% of QBI):", qbid_max)

# 3. Wage / UBIA limitation (non-SSTBs only) ----------------------------
qbi_non_sstb = (1 - is_sstb) * qbi
w2_wages_non_sstb = (1 - is_sstb) * w2_wages
ubia_property_non_sstb = (1 - is_sstb) * ubia_property

print("qbi_non_sstb:", qbi_non_sstb)
print("w2_wages_non_sstb:", w2_wages_non_sstb)
print("ubia_property_non_sstb:", ubia_property_non_sstb)

wage_limit = p.max.w2_wages.rate * w2_wages_non_sstb  # 50% of wages
print("wage_limit:", wage_limit)

alt_limit = (
    p.max.w2_wages.alt_rate * w2_wages_non_sstb  # 25% of wages
    + p.max.business_property.rate * ubia_property_non_sstb  # 2.5% of UBIA
)
print("alt_limit:", alt_limit)

wage_ubia_cap = max_(wage_limit, alt_limit)
print("wage_ubia_cap:", wage_ubia_cap)

step1_uncapped = p.max.rate * qbi_non_sstb
print("step1_uncapped:", step1_uncapped)

step1_deduction = min_(step1_uncapped, wage_ubia_cap)
print("step1_deduction:", step1_deduction)

# 4. Limitation phase-in amount (75% × excess) --------------------------
excess_income = max_(0, taxable_income - threshold)
print("excess_income:", excess_income)

phase_in_amount = phase_in_rate * excess_income
print("phase_in_amount (reduction):", phase_in_amount)

step2_deduction = max_(0, qbid_max - phase_in_amount)
print("step2_deduction:", step2_deduction)

# 5. QBI component: greater of Step 1 or Step 2 -------------------------
qbi_component = max_(step1_deduction, step2_deduction)
print("qbi_component:", qbi_component)

# 6. REIT, PTP, and BDC income (always 23%) -----------------------------
reit_ptp_bdc_base = reit_ptp_income + where(
    p_ref.use_bdc_income,
    bdc_income,
    0,
)
print("reit_ptp_bdc_base (RPB):", reit_ptp_bdc_base)

reit_ptp_bdc_deduction = p.max.rate * reit_ptp_bdc_base
print("reit_ptp_bdc_deduction:", reit_ptp_bdc_deduction)

total_before_income_cap = qbi_component + reit_ptp_bdc_deduction
print("total_before_income_cap:", total_before_income_cap)

# 7. Overall 23% taxable income ceiling ---------------------------------
income_cap = p.max.rate * taxable_income
print("income_cap (23% of taxable_income):", income_cap)

_return = min_(total_before_income_cap, income_cap)
print("Final QBID (_return):", _return)


phase_in_rate: 0.75
qbid_max (23% of QBI): 22000.0
qbi_non_sstb: 0
w2_wages_non_sstb: 0
ubia_property_non_sstb: 0
wage_limit: 0.0
alt_limit: 0.0
wage_ubia_cap: 0.0
step1_uncapped: 0.0
step1_deduction: 0.0
excess_income: 60000
phase_in_amount (reduction): 45000.0
step2_deduction: 0.0
qbi_component: 0.0
reit_ptp_bdc_base (RPB): 0
reit_ptp_bdc_deduction: 0.0
total_before_income_cap: 0.0
income_cap (23% of taxable_income): 80020.0
Final QBID (_return): 0.0
